In [1]:
############################################################################################################
## 1. 패키지 import
############################################################################################################
import pandas as pd
import sqlite3
import os
from sqlalchemy import create_engine
from datetime import datetime, timedelta
import time
from dateutil.relativedelta import relativedelta

# 사용 예시
from module.logger import log_message
import module.sqlTransaction as sqlT
import importlib
importlib.reload(sqlT)

############################################################################################################
## 2. 초기설정
############################################################################################################
# 경로설정
dir_work = f'c:/Users/user/OneDrive - 파인트리파트너스(주)/movie'
dir_func = f'{dir_work}/src/module'
dir_data = f'{dir_work}/data'
dir_srcmov = f'{dir_work}/data/srcmov'

# DB연결
conn = sqlite3.connect(f"{dir_data}/pine_movie.db", isolation_level=None)
cur = conn.cursor()

# 함수 호출
# exec(open(f"{dir_func}/sqlTransaction.py"      , encoding= 'utf-8').read() )
# exec(open(f"{dir_func}/logger.py"              , encoding= 'utf-8').read() )

# 1. 테이블 DDL

In [2]:
############################################################################################################
## 1. 테이블 DDL
############################################################################################################
# 캐스팅 정보
cur.execute("""drop table if exists CAST""")
cur.execute("""
    create table if not exists CAST (
          캐스팅번호              	
        , 이름                 
        , PRIMARY KEY(캐스팅번호)
    )
    """
)

# 크루 정보
cur.execute("""drop table if exists CREW""")
cur.execute("""
create table if not exists CREW (
	  크루번호                	
	, 이름                 
	, PRIMARY KEY(크루번호)
    )
    """
)

# 고객 정보
cur.execute("""drop table if exists CUSTOMER""")
cur.execute("""
create table if not exists CUSTOMER (
 	  회원번호                	
 	, 성                 
 	, 이름                 
 	, 도로명주소                 
 	, 우편번호                 
 	, 마을번호                 
 	, 마을명                 
 	, 주번호                 
 	, 주명                 
 	, 국가번호                 
 	, 국가명                 
 	, 대륙번호                 
 	, 대륙명                 
 	, 나이                 
 	, 통근거리                 
 	, 신용잔액                 
 	, 학력                 
 	, 이메일                 
 	, 고용유형                 
 	, 성별                 
 	, 가구규모                 
 	, 수입                 
 	, 수입구간                 
 	, 잔고부족발생건수                 
 	, 직장유형                 
 	, 부동산담보대출및임대지불지연건수                 
 	, 혼인유형                 
 	, 부동산담보대출금액                 
 	, 보유차량수                
 	, 부동산담보대출건수                 
 	, 애완동물유형                 
 	, 프로모션반응여부                 
 	, 주거유형                 
 	, 세그먼트번호                 
 	, 직장경험                 
 	, 현직장근속년수        
 	, 회원유지기간
 	, 거주기간                 
 	, 데이터베이스주소코드                
 	, 국가코드                 
 	, 유저코드            
 	, PRIMARY KEY(회원번호)
    )
    """
 )

# 고객 세그먼트 정보
cur.execute("""drop table if exists CUSTOMER_SERGMENT""")
cur.execute("""
create table if not exists CUSTOMER_SERGMENT (
	  세그먼트번호                	
	, 세그먼트명                 
	, 세그먼트정보
 	, PRIMARY KEY(세그먼트번호)
    )
    """
)

# 장르 정보
cur.execute("""drop table if exists GENRE""")
cur.execute("""
create table if not exists GENRE (
	  장르번호                	
	, 장르명 
 	, PRIMARY KEY(장르번호)
    )
    """
)

# 영화 정보
cur.execute("""drop table if exists MOVIE""")
cur.execute("""
create table if not exists MOVIE (
	  영화번호                	
	, 영화명 
	, 개봉년 
	, 제작비 
	, 박스오피스
    , 줄거리 
 	, PRIMARY KEY(영화번호)
    )
    """
)

# 영화 캐스팅 정보
cur.execute("""drop table if exists MOVIE_CAST""")
cur.execute("""
create table if not exists MOVIE_CAST (
	  영화번호                	
	, 캐스팅번호 
 	, PRIMARY KEY(영화번호, 캐스팅번호)
    )
    """
)

# 영화 크루 정보
cur.execute("""drop table if exists MOVIE_CREW""")
cur.execute("""
create table if not exists MOVIE_CREW (
	  영화번호                	
	, 크루번호 
    , 담당역할
 	, PRIMARY KEY(영화번호, 크루번호, 담당역할)
    )
    """
)

# 회원 행동 정보
cur.execute("""drop table if exists MOVIE_FACT""")
cur.execute("""
create table if not exists MOVIE_FACT (
	  회원번호                	
	, 영화번호 
	, 장르번호 
	, 기준년월일 
	, 추천여부 
	, 행동번호 
	, 평점 
	, 판매액
    , 기준년월 
 	, PRIMARY KEY(회원번호, 영화번호, 행동번호, 기준년월일, 추천여부, 장르번호)
    )
    """
)

# 영화 장르 정보
cur.execute("""drop table if exists MOVIE_GENRE""")
cur.execute("""
create table if not exists MOVIE_GENRE (
	  영화번호                	
	, 장르번호 
 	, PRIMARY KEY(영화번호, 장르번호)
    )
    """
)

# 고객 행동 유형 정보
cur.execute("""drop table if exists ACTIVITY""")
cur.execute("""
create table if not exists ACTIVITY (
	  행동번호                	
	, 행동명 
 	, PRIMARY KEY(행동번호)
    )
    """
)

# 2. 테이블 적재

In [3]:
############################################################################################################
## 2. 테이블 적재
############################################################################################################
####################################################################################
## 2-1. CSV 추출
####################################################################################
# file경로 내 파일리스트 인식
list_csv = os.listdir(dir_srcmov)

# 테이블 리스트 생성 및 데이터프레임 할당 반복
for i, x in enumerate(list_csv) :
    print(f"[LOG] 순서 = {i}, 추출파일 = {x.split('.')[0].lower()}")
    globals()['df_{}_origin'.format(x.split('.')[0].lower())] = pd.read_csv(rf"{dir_srcmov}\{x}")
    
####################################################################################
## 2-2. 테이블 컬럼명 한글 변환
####################################################################################
# 캐스팅 테이블 컬럼명 한글변환
df_cast = (
    df_cast_origin[[x for x in df_cast_origin.columns if "Unnamed" not in x]]
        .rename(
            columns = {
                  'CAST_ID'                     : '캐스팅번호'
                , 'NAME'                        : '이름'
            }
        )
)

# 크루 테이블 컬럼명 한글변환
df_crew = (
    df_crew_origin[[x for x in df_crew_origin.columns if "Unnamed" not in x]]
        .rename(
            columns = {
                  'CREW_ID'                      : '크루번호'
                , 'NAME'                         : '이름'
            }
        )
)

# 회원 테이블 컬럼명 한글변환
df_cust = (
    df_customer_origin[[x for x in df_customer_origin.columns if "Unnamed" not in x]]
        .rename(
            columns = {
                  'CUST_ID'                      : '회원번호'
                , 'LAST_NAME'                    : '성'
                , 'FIRST_NAME'                   : '이름'
                , 'STREET_ADDRESS'               : '도로명주소'
                , 'POSTAL_CODE'                  : '우편번호'
                , 'CITY_ID'                      : '마을번호'
                , 'CITY'                         : '마을명'
                , 'STATE_PROVINCE_ID'            : '주번호'
                , 'STATE_PROVINCE'               : '주명'
                , 'COUNTRY_ID'                   : '국가번호'
                , 'COUNTRY'                      : '국가명'
                , 'CONTINENT_ID'                 : '대륙번호'
                , 'CONTINENT'                    : '대륙명'
                , 'AGE'                          : '나이'
                , 'COMMUTE_DISTANCE'             : '통근거리'
                , 'CREDIT_BALANCE'               : '신용잔액'
                , 'EDUCATION'                    : '학력'
                , 'EMAIL'                        : '이메일'
                , 'FULL_TIME'                    : '고용유형'
                , 'GENDER'                       : '성별'
                , 'HOUSEHOLD_SIZE'               : '가구규모'
                , 'INCOME'                       : '수입'
                , 'INCOME_LEVEL'                 : '수입구간'
                , 'INSUFF_FUNDS_INCIDENTS'       : '잔고부족발생건수'
                , 'JOB_TYPE'                     : '직장유형'
                , 'LATE_MORT_RENT_PMTS'          : '부동산담보대출및임대지불지연건수'
                , 'MARITAL_STATUS'               : '혼인유형'
                , 'MORTGAGE_AMT'                 : '부동산담보대출금액'
                , 'NUM_CARS'                     : '보유차량수'
                , 'NUM_MORTGAGES'                : '부동산담보대출건수'
                , 'PET'                          : '애완동물유형'
                , 'PROMOTION_RESPONSE'           : '프로모션반응여부'
                , 'RENT_OWN'                     : '주거유형'
                , 'SEG'                          : '세그먼트번호'
                , 'WORK_EXPERIENCE'              : '직장경험'
                , 'YRS_CURRENT_EMPLOYER'         : '현직장근속년수'
                , 'YRS_CUSTOMER'                 : '회원유지기간'
                , 'YRS_RESIDENCE'                : '거주기간'
                , 'ADDRESS_POINT'                : '데이터베이스주소코드'
                , 'COUNTRY_CODE'                 : '국가코드'
                , 'USERNAME'                     : '유저코드'
            }
        )
)

# 고객 세그먼트 테이블 컬럼명 한글변환
df_cust_sgmt = (
    df_customer_segment_origin[[x for x in df_customer_segment_origin.columns if "Unnamed" not in x]]
        .rename(
            columns = {
                   'SEGMENT_ID'                  : '세그먼트번호'
                ,  'NAME'                        : '세그먼트명'
                ,  'DESCRIPTION'                 : '세그먼트정보'
            }
        )
)

# 장르 테이블 컬럼명 한글변환
df_genre = (
    df_genre_origin[[x for x in df_genre_origin.columns if "Unnamed" not in x]]
        .rename(
            columns = {
                  'GENRE_ID'                    : '장르번호'
                , 'NAME'                        : '장르명'
            }
        )
)

# 영화 테이블 컬럼명 한글변환
df_movie = (
    df_movie_origin[[x for x in df_movie_origin.columns if "Unnamed" not in x]]
        .rename(
            columns = {
                  'MOVIE_ID'                    : '영화번호'
                , 'TITLE'                       : '영화명'
                , 'YEAR'                        : '개봉년'
                , 'BUDGET'                      : '제작비'
                , 'GROSS'                       : '박스오피스'
                , 'PLOT_SUMMARY'                : '줄거리'
            }
        )
)

# 영화 캐스팅 테이블 컬럼명 한글변환
df_movie_cast = (
    df_movie_cast_origin[[x for x in df_movie_cast_origin.columns if "Unnamed" not in x]]
        .rename(
            columns = {
                  'MOVIE_ID'                    : '영화번호'
                , 'CAST_ID'                     : '캐스팅번호'
            }
        )
)

# 영화 크루 테이블 컬럼명 한글변환
df_movie_crew = (
    df_movie_crew_origin[[x for x in df_movie_crew_origin.columns if "Unnamed" not in x]]
        .rename(
            columns = {
                  'MOVIE_ID'                    : '영화번호'
                , 'CREW_ID'                     : '크루번호'
                , 'JOB'                         : '담당역할'
            }
        )
)

# 고객 행동 테이블 컬럼명 한글변환
df_movie_fact = (
    df_movie_fact_origin[[x for x in df_movie_fact_origin.columns if "Unnamed" not in x]]
        .rename(
            columns = {
                  'CUST_ID'                    : '회원번호'
                , 'MOVIE_ID'                   : '영화번호'
                , 'GENRE_ID'                   : '장르번호'
                , 'TIME_ID'                    : '기준년월일'
                , 'RECOMMENDED'                : '추천여부'
                , 'ACTIVITY_ID'                : '행동번호'
                , 'RATING'                     : '평점'
                , 'SALES'                      : '판매액'
                , 'BASEYM'                     : '기준년월'
            }
        )
)

# 영화 장르 테이블 컬럼명 한글변환
df_movie_genre = (
    df_movie_genre_origin[[x for x in df_movie_genre_origin.columns if "Unnamed" not in x]]
        .rename(
            columns = {
                  'MOVIE_ID'                   : '영화번호'
                , 'GENRE_ID'                   : '장르번호'
             
            }
        )
)

# 고객 행동 유형 테이블 컬럼명 한글변환
df_activity = (
    df_activity_origin[[x for x in df_activity_origin.columns if "Unnamed" not in x]]
        .rename(
            columns = {
                  'ACTIVITY_ID'               : '행동번호'
                , 'NAME'                      : '행동명'
             
            }
        )
)

############################################################################################################
## 2-3. DB 적재
############################################################################################################
# 캐스팅 테이블 데이터 초기화 - 데이터 적재 - 적재건수 확인
tblName = 'CAST'
df_temp = df_cast
pk = ['캐스팅번호']
cur.execute(f"""delete from {tblName}""")
sqlT.df_to_table(df_temp, pk, tblName, 10000)
sqlT.check_count(df_temp, pk, tblName)

# 크루 테이블 데이터 초기화 - 데이터 적재 - 적재건수 확인
tblName = 'CREW'
df_temp = df_crew
pk = ['크루번호']
cur.execute(f"""delete from {tblName}""")
sqlT.df_to_table(df_temp, pk, tblName, 10000)
sqlT.check_count(df_temp, pk, tblName)

# 회원 테이블 데이터 초기화 - 데이터 적재 - 적재건수 확인
tblName = 'CUSTOMER'
df_temp = df_cust
pk = ['회원번호']
cur.execute(f"""delete from {tblName}""")
sqlT.df_to_table(df_temp, pk, tblName, 100)
sqlT.check_count(df_temp, pk, tblName)


# 회원 테이블 데이터 초기화 - 데이터 적재 - 적재건수 확인
tblName = 'CUSTOMER_SERGMENT'
df_temp = df_cust_sgmt
pk = ['세그먼트번호']
cur.execute(f"""delete from {tblName}""")
sqlT.df_to_table(df_temp, pk, tblName, 10000)
sqlT.check_count(df_temp, pk, tblName)

# 장르 테이블 데이터 초기화 - 데이터 적재 - 적재건수 확인
tblName = 'GENRE'
df_temp = df_genre
pk = ['장르번호']
cur.execute(f"""delete from {tblName}""")
sqlT.df_to_table(df_temp, pk, tblName, 10000)
sqlT.check_count(df_temp, pk, tblName)

# 영화 테이블 데이터 초기화 - 데이터 적재 - 적재건수 확인
tblName = 'MOVIE'
df_temp = df_movie
pk = ['영화번호']
cur.execute(f"""delete from {tblName}""")
sqlT.df_to_table(df_temp, pk, tblName, 100)
sqlT.check_count(df_temp, pk, tblName)

# 영화 캐스팅 테이블 데이터 초기화 - 데이터 적재 - 적재건수 확인
tblName = 'MOVIE_CAST'
df_temp = df_movie_cast
pk = ['영화번호', '캐스팅번호']
cur.execute(f"""delete from {tblName}""")
sqlT.df_to_table(df_temp, pk, tblName, 10000)
sqlT.check_count(df_temp, pk, tblName)

# 영화 크루 테이블 데이터 초기화 - 데이터 적재 - 적재건수 확인
tblName = 'MOVIE_CREW'
df_temp = df_movie_crew
pk = ['영화번호', '크루번호', '담당역할']
cur.execute(f"""delete from {tblName}""")
sqlT.df_to_table(df_temp, pk, tblName, 10000)
sqlT.check_count(df_temp, pk, tblName)

# 회원 행동 테이블 데이터 초기화 - 데이터 적재 - 적재건수 확인
tblName = 'MOVIE_FACT'
df_temp = df_movie_fact
pk = ['회원번호','영화번호','행동번호', '추천여부', '장르번호', '기준년월일']
cur.execute(f"""delete from {tblName}""")
sqlT.df_to_table(df_temp, pk, tblName, 2000)
sqlT.check_count(df_temp, pk, tblName)

# 영화 장르 테이블 데이터 초기화 - 데이터 적재 - 적재건수 확인
tblName = 'MOVIE_GENRE'
df_temp = df_movie_genre
pk = ['영화번호', '장르번호']
cur.execute(f"""delete from {tblName}""")
sqlT.df_to_table(df_temp, pk, tblName, 10000)
sqlT.check_count(df_temp, pk, tblName)

# 행동 테이블 데이터 초기화 - 데이터 적재 - 적재건수 확인
tblName = 'ACTIVITY'
df_temp = df_activity
pk = ['행동번호']
cur.execute(f"""delete from {tblName}""")
sqlT.df_to_table(df_activity, pk, tblName, 10000)
sqlT.check_count(df_temp, pk, tblName)


[LOG] 순서 = 0, 추출파일 = activity
[LOG] 순서 = 1, 추출파일 = cast
[LOG] 순서 = 2, 추출파일 = crew
[LOG] 순서 = 3, 추출파일 = customer
[LOG] 순서 = 4, 추출파일 = customer_segment
[LOG] 순서 = 5, 추출파일 = genre
[LOG] 순서 = 6, 추출파일 = movie
[LOG] 순서 = 7, 추출파일 = movie_cast
[LOG] 순서 = 8, 추출파일 = movie_crew
[LOG] 순서 = 9, 추출파일 = movie_fact
[LOG] 순서 = 10, 추출파일 = movie_genre
[LOG] 대상테이블 = CAST | DB 및 DF 적재 건수 탐색 시작
[LOG] 대상테이블 = CAST | DB적재건수   = 10401
[LOG] 대상테이블 = CAST | DF적재건수   = 10401
[LOG]  -------------------------------------------------------------------
[LOG] 대상테이블 = CREW | DB 및 DF 적재 건수 탐색 시작
[LOG] 대상테이블 = CREW | DB적재건수   = 6859
[LOG] 대상테이블 = CREW | DF적재건수   = 6859
[LOG]  -------------------------------------------------------------------
[LOG] 대상테이블 = CUSTOMER | DB 및 DF 적재 건수 탐색 시작
[LOG] 대상테이블 = CUSTOMER | DB적재건수   = 4749
[LOG] 대상테이블 = CUSTOMER | DF적재건수   = 4749
[LOG]  -------------------------------------------------------------------
[LOG] 대상테이블 = CUSTOMER_SERGMENT | DB 및 DF 적재 건수 탐색 시작
[LOG] 대상테이블 = CUSTOMER_SERGM

{'DB건수': 11, 'DF건수': 11}

[LOG] 대상테이블 = CAST | 전체 테이블 데이터 추출 시작
